# Het verschil tussen startpositie en strategie

In [1]:
from IPython.display import HTML

# Create the HTML content as a string
html_content = """
<div id="f1-heatmap-container">
    <style>
        #f1-heatmap-container {
            font-family: Arial, sans-serif;
            background: white;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
            max-width: 1200px;
        }
        .controls {
            margin-bottom: 20px;
            padding: 15px;
            background: #f8f9fa;
            border-radius: 5px;
        }
        .controls select {
            padding: 8px 12px;
            font-size: 14px;
            border: 1px solid #ddd;
            border-radius: 4px;
            margin-left: 10px;
        }
        .info-box {
            background: #e3f2fd;
            padding: 10px;
            border-radius: 5px;
            margin: 10px 0;
            border-left: 4px solid #2196f3;
        }
        #heatmap {
            width: 100%;
            height: 700px;
        }
        .loading {
            text-align: center;
            padding: 50px;
            color: #666;
        }
    </style>
    
    <h2>🏎️ F1 Starting vs Finishing Position Analysis</h2>
    
    <div class="controls">
        <label for="yearSelect"><strong>Select Year:</strong></label>
        <select id="yearSelect">
            <option value="total">All Years (Total)</option>
        </select>
    </div>
    
    <div id="infoBox" class="info-box">
        <div class="loading">Loading F1 data...</div>
    </div>
    
    <div id="heatmap"></div>
</div>

<script src="https://cdnjs.cloudflare.com/ajax/libs/plotly.js/2.32.0/plotly.min.js"></script>
<script>
    // Parse CSV data
    function parseCSV(csvText) {
        const lines = csvText.trim().split('\\n');
        const headers = lines[0].split(',').map(h => h.trim().replace(/"/g, ''));
        const data = [];
        
        for (let i = 1; i < lines.length; i++) {
            const values = lines[i].split(',').map(v => v.trim().replace(/"/g, ''));
            if (values.length === headers.length) {
                const row = {};
                headers.forEach((header, index) => {
                    if (header === 'Year' || header === 'Pos') {
                        row[header] = parseInt(values[index]);
                    } else {
                        row[header] = values[index];
                    }
                });
                data.push(row);
            }
        }
        return data;
    }

    // Load F1 data from CSV file
    async function loadF1Data() {
        try {
            const response = await fetch('merged_f1_data_1994_2022.csv');
            if (!response.ok) {
                throw new Error(`HTTP error! status: ${response.status}`);
            }
            const csvText = await response.text();
            return parseCSV(csvText);
        } catch (error) {
            console.error('Error loading F1 data:', error);
            return [];
        }
    }

    // Create heatmap data
    function createHeatmapData(data, selectedYear = null) {
        let filteredData = data;
        let titleSuffix = " - All Years (1994-2022)";
        
        if (selectedYear && selectedYear !== 'total') {
            filteredData = data.filter(d => d.Year === parseInt(selectedYear));
            titleSuffix = ` - ${selectedYear}`;
        }

        const maxStartPos = Math.max(...filteredData.map(d => d.Pos));
        const startPositions = Array.from({length: Math.min(maxStartPos, 26)}, (_, i) => i + 1);
        
        const allFinishPositions = [...new Set(filteredData.map(d => d.FinPos))];
        const numericPositions = [];
        const nonNumericPositions = [];
        
        allFinishPositions.forEach(pos => {
            const numPos = parseInt(pos);
            if (!isNaN(numPos) && numPos >= 1 && numPos <= 30) {
                numericPositions.push(numPos);
            } else if (pos && pos !== 'nan') {
                if (['DNF', 'DSQ', 'NC', 'DNS', 'EX', 'WD'].includes(pos)) {
                    if (!nonNumericPositions.includes('DNF')) {
                        nonNumericPositions.push('DNF');
                    }
                } else {
                    nonNumericPositions.push(pos);
                }
            }
        });
        
        const finishPositions = numericPositions.sort((a, b) => a - b).concat(nonNumericPositions.sort());
        
        const matrix = Array(finishPositions.length).fill().map(() => Array(startPositions.length).fill(0));
        
        filteredData.forEach(row => {
            const startIdx = startPositions.indexOf(row.Pos);
            if (startIdx === -1) return;
            
            let finishIdx = -1;
            const finishPos = row.FinPos;
            
            if (['DNF', 'DSQ', 'NC', 'DNS', 'EX', 'WD'].includes(finishPos)) {
                finishIdx = finishPositions.indexOf('DNF');
            } else {
                const numFinish = parseInt(finishPos);
                if (!isNaN(numFinish)) {
                    finishIdx = finishPositions.indexOf(numFinish);
                } else {
                    finishIdx = finishPositions.indexOf(finishPos);
                }
            }
            
            if (finishIdx !== -1) {
                matrix[finishIdx][startIdx]++;
            }
        });

        return {
            matrix,
            startPositions,
            finishPositions,
            titleSuffix,
            totalRecords: filteredData.length
        };
    }

    // Create Plotly heatmap
    function createHeatmap(data, selectedYear = null) {
        const { matrix, startPositions, finishPositions, titleSuffix, totalRecords } = createHeatmapData(data, selectedYear);
        
        const hoverText = matrix.map((row, i) => 
            row.map((count, j) => {
                const percentage = totalRecords > 0 ? (count / totalRecords * 100).toFixed(1) : 0;
                return `Starting: P${startPositions[j]}<br>Finishing: ${finishPositions[i]}<br>Count: ${count}<br>Percentage: ${percentage}%`;
            })
        );

        const trace = {
            z: matrix,
            x: startPositions.map(p => `P${p}`),
            y: finishPositions.map(p => p.toString()),
            type: 'heatmap',
            colorscale: 'RdYlBu',
            reversescale: true,
            hoverongaps: false,
            hovertemplate: '%{customdata}<extra></extra>',
            customdata: hoverText,
            colorbar: {
                title: "Number of Occurrences"
            }
        };

        const layout = {
            title: {
                text: `F1 Starting vs Finishing Position Heatmap${titleSuffix}<br><sub>Total Records: ${totalRecords.toLocaleString()}</sub>`,
                x: 0.5,
                font: { size: 16 }
            },
            xaxis: {
                title: 'Starting Position',
                side: 'bottom'
            },
            yaxis: {
                title: 'Finishing Position',
                autorange: 'reversed'
            },
            width: 1000,
            height: 700,
            font: { size: 12 },
            margin: { t: 100, b: 60, l: 80, r: 80 }
        };

        Plotly.newPlot('heatmap', [trace], layout, {responsive: true});
    }

    // Update info box
    function updateInfo(data, selectedYear) {
        const infoBox = document.getElementById('infoBox');
        let filteredData = data;
        
        if (selectedYear && selectedYear !== 'total') {
            filteredData = data.filter(d => d.Year === parseInt(selectedYear));
        }
        
        const recordsCount = filteredData.length;
        const racesCount = new Set(filteredData.map(d => d['Grand Prix'])).size;
        
        if (selectedYear === 'total') {
            infoBox.innerHTML = `📊 Showing data for ALL YEARS: ${recordsCount.toLocaleString()} total records`;
        } else {
            infoBox.innerHTML = `📊 Showing data for ${selectedYear}: ${recordsCount.toLocaleString()} records from ${racesCount} races`;
        }
    }

    // Initialize when this script runs
    (async function() {
        const f1Data = await loadF1Data();
        
        if (f1Data.length === 0) {
            document.getElementById('infoBox').innerHTML = '❌ Error: Could not load F1 data. Make sure "merged_f1_data_1994_2022.csv" is in the same directory as this notebook.';
            return;
        }
        
        const years = [...new Set(f1Data.map(d => d.Year))].sort((a, b) => a - b);
        const yearSelect = document.getElementById('yearSelect');
        
        yearSelect.innerHTML = '<option value="total">All Years (Total)</option>';
        
        years.forEach(year => {
            const option = document.createElement('option');
            option.value = year;
            option.textContent = year;
            yearSelect.appendChild(option);
        });

        yearSelect.addEventListener('change', function() {
            const selectedYear = this.value;
            updateInfo(f1Data, selectedYear);
            createHeatmap(f1Data, selectedYear === 'total' ? null : selectedYear);
        });

        updateInfo(f1Data, 'total');
        createHeatmap(f1Data, null);
    })();
</script>
"""

# Display the HTML
HTML(html_content)